# Doc: Labeling algorithm

Here the process of labeling the text extracted from the pdf and html files will be described starting from the output of the functions in html_layout_analysis.py,	pdf_layout_analysis.py in the nationbetter package.

In [1]:
import pandas as pd
import pickle
import os, re
from geotext import GeoText
from flashtext import KeywordProcessor

<font color='red'><b> TODO: store filename in pd.DataFrame.name varialbe!  </b></font>

<font color='red'><b>TODO: It seems some footers are still present! Go back and delete them! </b><br> The files are :<br>[0] Tier 2 Policy Guidance<br>
[5] Nationality: good character requirement<br>
[8] Appendix B - IMMIGRATION OFFENCES<br></font>

<font color='red'><b>TODO: Delete the header on title page of "Tier 2 of the Points Based System – Policy Guidance" in nationbetter.pdf_layout_analysis.get_lines_and_info() - In fact tiltes are not detected correctly </b> </font>

<font color='red'><b>Note on pdf_layout_analysis.py output</b>: To streamline the development process the output of the nationbetter package dumped in nationbetter_data/formatted_html_dfs/ is a simplified version of the full information extracted from the pdfs by the functions in pdf_layout_analysis.py. Line numbers, positions of the text on the page (see notebook for the nationbetter.pdf_layout_analysis.get_lines_and_info() function) and fontsizes were originally extraced also extracted and contained in the dataframes in nationbetter_data/formatted_html_dfs/ but were dropped to simplify formatting in for the labeling stage. Other layout functions were also disabled which used to extract the urls and tables as well as their position. Unfortunately due to the structure of pdfminer we were unable to directly extract the text the table and url objects because it seems the text which they refer to is stored in a different layer named TextBox. Initially there were plans to retrieve the text of these objects by matching their positions with those of the lines extracted in the pdf_layout_analysis.py but due to shifting priority to other developement issues these features were abandoned. Therefore besides dropping the respective columns at the end of the get_lines_and_info() function in pdf_layout_analysis.py the functions written in nationbetter/pdf_page_layout.py and nationbetter/pdf_page_parsing.py (responsible for extracting the urls and tables) remain unused in the package.</font>

## Importing files from the formatted_html_dfs and formatted_pdf_dfs in nationbetter_data/

In [2]:
# Here put the path to the nationbetter_data folder written by the file_handler in the 
# examples/example_build_data.py file in the nationbetter/ package
output_path =  os.path.abspath(os.path.join('..','..','..','nationbetter_data'))
formatted_html_path = os.path.join(output_path,'formatted_html_dfs')
formatted_pdf_path = os.path.join(output_path,'formatted_pdf_dfs')
formatted_html_pkls = os.listdir(formatted_html_path)
formatted_pdf_pkls = os.listdir(formatted_pdf_path)

For now all files will be extracted and keyed separately. Here some examples from of the output from the formatted extraction algorithm.

In [3]:
sample_html_df = pickle.load(open(os.path.join(formatted_html_path,formatted_html_pkls[1]),'rb'))
sample_pdf_df = pickle.load(open(os.path.join(formatted_pdf_path,formatted_pdf_pkls[5]),'rb'))

In [4]:
sample_html_df

,section,subsection,is table,string
0,0,0,False,Immigration Rules Appendix 7: overseas workers...
1,0,1,False,Overseas Domestic Worker
2,0,1,False,Employment contract\nAppendix 7 (MS Word versi...
3,0,1,True,...


In [68]:
# #load all pdfs
# pdfs_raw =[]
# for file in formatted_pdf_pkls:
#     df = pickle.load(open(os.path.join(formatted_pdf_path,file),'rb'))
#     pdfs_raw.append(df)
# namelist = []
# for df in pdfs_raw:
#     text_ser = df['text_type']
#     title_loc = df[text_ser=='title'].index.min()
#     namelist.append(df.text.replace('\s+', ' ', regex=True).str.strip()[title_loc])
# namelist
# pdfs_raw[0]
sample_pdf_df

,title,subtitle,text_type,page_no,text,url
0,0.0,0.0,title,1,\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n ...,https://assets.publishing.service.gov.uk/gover...
1,0.0,0.0,main_text,1,\nVersion 04/2019 \n \n \n \n \n \n \n \n \n ...,https://assets.publishing.service.gov.uk/gover...
2,1.0,0.0,title,3,\n \n \nContents \n,https://assets.publishing.service.gov.uk/gover...
3,1.0,0.0,main_text,4,Contents ........................................,https://assets.publishing.service.gov.uk/gover...
4,2.0,0.0,title,5,\n \nAbout this guidance \n,https://assets.publishing.service.gov.uk/gover...
...,...,...,...,...,...,...
93,12.0,1.0,main_text,29,100. You can undertake work that is supplemen...,https://assets.publishing.service.gov.uk/gover...
94,12.0,2.0,subtitle,29,\nChange of employment \n,https://assets.publishing.service.gov.uk/gover...
95,12.0,2.0,main_text,29,104. If you have previously received permissi...,https://assets.publishing.service.gov.uk/gover...
96,12.0,3.0,subtitle,29,\n \n \nOther changes \n,https://assets.publishing.service.gov.uk/gover...


# Extracting section titles (HTML)

Due to the more structured nature of the HTML documents (only sections and subsections appear), no extra care has to be taken with the keys and the section titles can be extracted as follows

In [6]:
#load all htmls
htmls =[]
for file in formatted_html_pkls:
    df = pickle.load(open(os.path.join(formatted_html_path,file),'rb'))
    htmls.append(df)

In [7]:
def get_title_col_html(df,sec_col='string'):
    #Checking if the (sub)section title changes if yes return true
    is_sec_title = (df[sec_col]-df[sec_col].shift().bfill()).astype(bool)
    colname = sec_col + '_title'
    #Selects string column where is_sec_title true, returns section name
    df[colname]= df[is_sec_title].string
    df[colname].fillna(method='ffill',inplace = True)

def agg_subsections_html(df):
    #get columns which need to be added 
    list_to_new_keys = ['section','subsection']
    for key in list_to_new_keys:
        get_title_col_html(df,key)
    df.groupby(['section','subsection'],as_index = False).agg({'string':' '.join,'section_title':'first','subsection_title':'first'},inplace = True)

for df in htmls:
    agg_subsections_html(df)
    
    

In [8]:
htmls[1]

,section,subsection,is table,string,section_title,subsection_title
0,0,0,False,Immigration Rules Appendix 7: overseas workers...,NaN,NaN
1,0,1,False,Overseas Domestic Worker,NaN,Overseas Domestic Worker
2,0,1,False,Employment contract\nAppendix 7 (MS Word versi...,NaN,Overseas Domestic Worker
3,0,1,True,...,NaN,Overseas Domestic Worker


# Cleanup pdf 2.0

In some cases of the PDFs a non regular header appears on the first page. since this messes up the counting and labeling of the sections we delete it here. This might need to be implemented on the pdf reader level. We do it here as a temporary fix.

In [238]:
#load all pdfs
pdfs =[]
for file in formatted_pdf_pkls:
    df = pickle.load(open(os.path.join(formatted_pdf_path,file),'rb'))
    pdfs.append(df)
    df[df['page_no']==1]

for df in pdfs:
    title_idx =df[(df['text_type']=='title')&(df['page_no']==1)].index[0]
    has_sub = df[(df['text_type']=='subtitle')&(df['page_no']==1)].index
    if title_idx != 0:
        df.drop([title_idx-1],inplace = True)
    if list(has_sub):
        if has_sub[0]>title_idx:
            new_title= df.iloc[title_idx].text + df.iloc[has_sub[0]].text
            df.at[0,'text']= new_title


Since the PDFs are less structured and have different titles for the section and subsection columns (title and subtitle respectively) they have to be renamed. Also sometimes these labels have a more shallow or deeper structure e.g. only title or title subtitle subsubtitle etc. To do the same sectioning as the html we will first rename all the columns to (sub) section and then automate extraction of the section names and the aggregation of the text columns  to work over this variable set of column names.

In [239]:
# This part of the code fixes a bug dropping the "empty" subsection counter columns in 
# pdfs[0] = Tier 2 Policy Guidance. There are however more issues and this fix should
# be implemented on a deeper leve
keylist = []
for df in pdfs:
    keylist.append(df.keys())
keylist
#dropping empty (sub)title columns
for df_no in range(len(pdfs)):
    find_sec = re.compile(r'\w*title$')
    find=list(filter(None,[find_sec.search(key) for key in keylist[df_no]]))
    old_keys = [key.group() for key in find]
    for key in old_keys:
        if len(pdfs[df_no][key].unique())==1:
            pdfs[df_no].drop(key,axis =1,inplace=True)
#rebuild keylist for new columns
keylist = []
for df in pdfs:
    keylist.append(df.keys())

In [240]:
#renaming the columns in the dataframe with the convention of html 
for df_no in range(len(pdfs)):
    find_sec = re.compile(r'\w*title$')
    find=list(filter(None,[find_sec.search(key) for key in keylist[df_no]]))
    old_keys = [key.group() for key in find]
    #if the dataframe of pdfs[0] is fixed and keys still need to be dropped
    # insert here some code that renames missing keys and columns
    # ['title','subsubsection'] -> ['title','subtitle']
    keep_keys = keylist[df_no].drop(old_keys)
    find_sec = re.compile(r'(title)$')
    final_keys = [find_sec.sub(r'section',key) for key in keylist[df_no]]
    rename_keys = pd.Index(final_keys).drop(keep_keys)
    rename_dict = dict(zip(old_keys,rename_keys))
    pdfs[df_no].rename(columns = rename_dict,inplace = True)

In [241]:
pdfs[7]

,section,subsection,subtext,text_type,page_no,text,url
0,0.0,0.0,0.0,title,1,\n \n \n \n \n \n \n \n \n \nGuide AN \nNatur...,https://assets.publishing.service.gov.uk/gover...
1,0.0,0.0,0.0,main_text,1,\n \nJuly 2020 \n,https://assets.publishing.service.gov.uk/gover...
2,1.0,0.0,0.0,title,1,\n \n \n \n \n \n \nContents \n,https://assets.publishing.service.gov.uk/gover...
3,1.0,0.0,0.0,main_text,2,Naturalisation Booklet – The Requirements .......,https://assets.publishing.service.gov.uk/gover...
4,1.0,0.0,1.0,subtext,2,The residence requirements: .....................,https://assets.publishing.service.gov.uk/gover...
...,...,...,...,...,...,...,...
179,16.0,12.0,0.0,main_text,41,A letter from the relevant employer confirming...,https://assets.publishing.service.gov.uk/gover...
180,16.0,12.0,1.0,subtext,41,\n \n40 \n,https://assets.publishing.service.gov.uk/gover...
181,17.0,0.0,20.0,title,41,\nCitizenship Ceremonies \n,https://assets.publishing.service.gov.uk/gover...
182,17.0,0.0,20.0,main_text,42,\nIf your application is successful and you a...,https://assets.publishing.service.gov.uk/gover...


In [242]:
#write function that can be iterated over to obtain section title columns
#for single df
# df_no = 1
# Sorting the keys into agg_keys, which will generate the column names
# and dropping unwanted keys
keylist = []
for df in pdfs:
    keylist.append(df.keys())

def get_title_col_pdf(df,sec_col='string'):
    '''
    Taking a column (of section counters) and check weither a section
    transition occurs. If yes store in column [(sub)*section]_title
    the tite (assuming it happpends) on the first line of the section
    '''
    #Checking if the (sub)section title changes if yes return true
    is_sec_title = (df[sec_col]-df[sec_col].shift().bfill()).astype(bool)
    colname = sec_col + '_title'  
    return is_sec_title, colname


for num, df in enumerate(pdfs):
    #get the keys
    keylist = df.keys()
    find_sec = re.compile(r'\w*section$')
    find=list(filter(None,[find_sec.search(key) for key in keylist]))
    agg_keys = [key.group() for key in find]
    sec_keys = agg_keys + ['page_no','text','url']
    #reduce the dataframe
    df = df[sec_keys]

    #call function
    for key in agg_keys:
        is_sec_title, colname = get_title_col_pdf(df,key)
        #Selects string column where is_sec_title true, returns section name
        titlecol = df[is_sec_title].text.replace('\s+', ' ', regex=True).str.strip()
        titlecol.name = colname
        df = pd.concat([df,titlecol],axis = 1)
        df[colname].fillna(method='ffill',inplace=True)

    #Generate dict for aggregation of text in (sub)*sections
    dex = df.keys().drop(agg_keys+['text']) 
    agg_fn = ['first' for no_dex in range(len(dex))]
    agg_arg = dict(zip(dex,agg_fn))
    agg_arg.update({'text':' '.join})
    df.groupby(agg_keys,as_index = False).agg(agg_arg)
    df.rename(columns={'text':'string'},inplace=True)
    pdfs[num]=df

In [243]:
pdfs[1]

,subsubsection,page_no,string,url,subsubsection_title
1,0.0,1,\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n ...,https://assets.publishing.service.gov.uk/gover...,NaN
2,0.0,1,\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n ...,https://assets.publishing.service.gov.uk/gover...,NaN
3,0.0,2,Tier 2 Policy Guidance (Version 07/2020) \nPage\n,https://assets.publishing.service.gov.uk/gover...,NaN
4,0.0,2,\n \n \n 2 \n,https://assets.publishing.service.gov.uk/gover...,NaN
5,0.0,3,Institution .....................................,https://assets.publishing.service.gov.uk/gover...,NaN
...,...,...,...,...,...
182,2.0,81,\nTier 2 Policy Guidance (Version 07/2020) \n...,https://assets.publishing.service.gov.uk/gover...,Resident labour market exemptions
183,2.0,81,\n• points have been correctly awarded; \n \...,https://assets.publishing.service.gov.uk/gover...,Resident labour market exemptions
184,2.0,82,\nTier 2 Policy Guidance (Version 07/2020) \n...,https://assets.publishing.service.gov.uk/gover...,Resident labour market exemptions
185,2.0,82,\nC13. Does Administrative Review cover veri...,https://assets.publishing.service.gov.uk/gover...,Resident labour market exemptions


# PDF extract the title & url

In [244]:
# takes pdfs, reads first line which should be title and stores it
namelist2 = []
for df in pdfs:
    title_loc = df.index.min()
    namelist2.append(df.string.replace('\s+', ' ', regex=True).str.strip()[title_loc])
namelist2

['Tiers 2, 4 and 5 of the points-based system: guidance for sponsors Appendix D: keeping documents - guidance for sponsors',
 'Tier 2 of the Points Based System – Policy Guidance',
 'Tier 2 and 5: Guidance for Sponsors - Addendum',
 'Indefinite leave to remain: calculating continuous period in UK',
 'English language requirements: skilled workers',
 'Tier 5 Temporary Worker of the Points Based System.',
 'Nationality: good character requirement',
 'Guide AN Naturalisation Booklet – The Requirements',
 'Nationality policy: Naturalisation as a British citizen by discretion',
 'Appendix B - IMMIGRATION OFFENCES']

In [248]:
url_list=[]
for df in pdfs:
    url_list.append(df['url'][1])
import pprint
pprint.pprint(dict(zip(namelist2,url_list)))

{'Appendix B - IMMIGRATION OFFENCES': 'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/255350/sponsorguideappBfrom060412.pdf',
 'English language requirements: skilled workers': 'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/870839/english-language-v17.0ext.pdf',
 'Guide AN Naturalisation Booklet – The Requirements': 'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/904395/Guide-AN_-_July_20.pdf',
 'Indefinite leave to remain: calculating continuous period in UK': 'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/891368/calculating-continuous-leave-v21.0-gov-uk.pdf',
 'Nationality policy: Naturalisation as a British citizen by discretion': 'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/885368/naturalisation-as-a-british-citizen-by-discretion-v5.0-g

In [249]:
namelist2[0]

'Tiers 2, 4 and 5 of the points-based system: guidance for sponsors Appendix D: keeping documents - guidance for sponsors'

In [250]:
url_list[0]

'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/878156/2020-04-03_Sponsor-guidance_Appendix-D_04-2020_v1.0.pdf'